<a href="https://colab.research.google.com/github/kdhdr1/WarmTransfer/blob/main/CNN_style_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
# 학습 완료한 파일 연동을 위한 drive mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from __future__ import print_function
import numpy as np
from PIL import Image
import time
 
from keras.applications.vgg16 import VGG16
 
from keras import backend
from keras.models import Model
from scipy.optimize import fmin_l_bfgs_b
 

height = 512
width = 512
 
# 이미지를 불러오고 사이즈를 바꿉니다.
content_image = Image.open('/content/drive/MyDrive/project/img/image/69996.png')
content_image = content_image.resize((width, height))
 
style_image = Image.open('/content/drive/MyDrive/project/img/image/source_6.jpg')
style_image = style_image.resize((width, height))
 
# 이미지의 픽셀 데이터를 얻어오고 차원을 하나 추가합니다.
# 만약 채널이 4개면 마지막 채널을 제거합니다.
content_array = np.asarray(content_image, dtype='float32')
if (content_array.shape[2] == 4):
  content_array = content_array[:, :, :-1]
content_array = np.expand_dims(content_array, axis=0)
 
style_array = np.asarray(style_image, dtype='float32')
if (style_array.shape[2] == 4):
  style_array = style_array[:, :, :-1]
style_array = np.expand_dims(style_array, axis=0)
 
print(content_array.shape)
print(style_array.shape)
 
# 이미지 데이터를 전처리 합니다.
def subtract_mean_rgb(img_arr):
  img_arr[:, :, :, 0] -= 103.939
  img_arr[:, :, :, 1] -= 116.779
  img_arr[:, :, :, 2] -= 123.68
  return img_arr[:, :, :, ::-1]
 
content_array = subtract_mean_rgb(content_array)
style_array = subtract_mean_rgb(style_array)
 
# 이미지 데이터를 케라스 변수로 만듭니다.
content_image = backend.variable(content_array)
style_image = backend.variable(style_array)
combination_image = backend.placeholder((1, height, width, 3))
 
# 신경망에 실제로 넣을 데이터를 만듭니다.
input_tensor = backend.concatenate([content_image, style_image, combination_image], axis=0)
print(input_tensor.shape)
 
# 모델을 불러옵니다.
model = VGG16(input_tensor=input_tensor, weights='imagenet', include_top=False)
 
# 스타일 변환 매개변수 선언.
content_weight = 0.05
style_weight = 5.0
total_variation_weight = 1.0
 
# 모델의 여러 레이어에서 값을 사용할 것이므로 미리 모든 레이어를 가져옴.
layers = dict([(layer.name, layer.output) for layer in model.layers])
 
# 전체 손실을 담을 변수 선언.
loss = backend.variable(0.0)
 
# Content 손실 함수(MSE)
def content_loss(content, combination):
  return backend.sum(backend.square(content - combination))
 
# Content 손실을 계산합니다.
# block2_conv2는 VGG의 2-2번째 합성곱 + Relu 계층입니다.
# 어떤 계층을 사용할지는 하이퍼 파리미터입니다.
layer_features = layers['block2_conv2']
content_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(content_image_features, combination_features)
 
# 이미지에서 스타일에만 집중하도록 그람 행렬을 사용합니다.
def gram_matrix(x):
  features = backend.batch_flatten(backend.permute_dimensions(x, (2, 0, 1)))
  gram = backend.dot(features, backend.transpose(features))
  return gram
 
# Style 손실 함수
def style_loss(style, combination):
  S = gram_matrix(style)
  C = gram_matrix(combination)
  channels = 3
  size = height * width
  st = backend.sum(backend.square(S - C)) / (4. * (channels ** 2) * (size ** 2))
  return st
 
# 스타일 손실을 계산할 계층
feature_layers = ['block1_conv2', 'block2_conv2',
                  'block3_conv3', 'block4_conv3',
                  'block5_conv3']
 
# 스타일 손실을 계산합니다.
for layer_name in feature_layers:
  layer_features = layers[layer_name]
  style_features = layer_features[1, :, :, :]
  combination_features = layer_features[2, :, :, :]
  sl = style_loss(style_features, combination_features)
  loss = loss + (style_weight / len(feature_layers)) * sl
 
# 결과 이미지의 노이즈를 조절하기 위한 손실 함수
def total_variation_loss(x):
  a = backend.square(x[:, :height-1, :width-1, :] - x[:, 1:, :width-1, :])
  b = backend.square(x[:, :height-1, :width-1, :] - x[:, :height-1, 1:, :])
  return backend.sum(backend.pow(a + b, 1.25))
loss = loss + total_variation_weight * total_variation_loss(combination_image)
 
# 학습을 위해 경사도를 계산합니다.
grads = backend.gradients(loss, combination_image)
 
# scipy.optimize는 Loss와 Grads를 따로 요구하는데
# 따로 계산하는건 비효율적이므로 아래 코드가 사용됨.
outputs = [loss]
if isinstance(grads, (list, tuple)):
  outputs += grads
else:
  outputs.append(grads)
f_outputs = backend.function([combination_image], outputs)
 
def eval_loss_and_grads(x):
  x = x.reshape((1, height, width, 3))
  outs = f_outputs([x])
  loss_value = outs[0]
  grad_values = outs[1].flatten().astype('float64')
  return loss_value, grad_values
 
class Evaluator(object):
  def __init__(self):
    self.loss_value = None
    self.grads_values = None
    
  def loss(self, x):
    assert self.loss_value is None
    loss_value, grad_values = eval_loss_and_grads(x)
    self.loss_value = loss_value
    self.grad_values = grad_values
    return self.loss_value
 
  def grads(self, x):
    assert self.loss_value is not None
    grad_values = np.copy(self.grad_values)
    self.loss_value = None
    self.grad_values = None
    return grad_values
  
evaluator = Evaluator()
 
# 노이즈 이미지 데이터 생성.
x = np.random.uniform(0, 255, (1, height, width, 3)) - 128.0
 
# 학습 진행.
iterations = 2
for i in range(iterations):
  print('Start of iteration', i)
  start_time = time.time()
  x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                   fprime=evaluator.grads, maxfun=20)
  print(min_val)
  end_time = time.time()
  print('Iteration %d completed in %ds' % (i, end_time - start_time))
  
# 결과 데이터를 이미지로 바꾸기 위한 처리.
x = x.reshape((height, width, 3))
x = x[:, :, ::-1]
x[:, :, 0] += 103.939
x[:, :, 1] += 116.779
x[:, :, 2] += 123.68
x = np.clip(x, 0, 255).astype('uint8')
 
result_img = Image.fromarray(x)
result_img.save("/content/drive/MyDrive/project/img/image/result_iter2_6_1.png")

(1, 512, 512, 3)
(1, 512, 512, 3)
(3, 512, 512, 3)
Start of iteration 0
74663780000.0
Iteration 0 completed in 15s
Start of iteration 1
30527721000.0
Iteration 1 completed in 14s


In [ ]:
result_img = result_img.resize((6798, 4625))

In [ ]:
result_img.save("/content/drive/MyDrive/project/img/image/result_iter2_6_1.png")